In [ ]:
import psycopg2
import pandas as pd
import geopandas as gpd
import pyproj
from pylab import *
import pdb

#caches a list of all the routing link elements (gids) for each known routeID, to allow faster lookup during count processing.

con = psycopg2.connect(database='dcc', user='charles')
cur = con.cursor()
wgs84  = pyproj.Proj(init='epsg:4326')  #WGS84
bng    = pyproj.Proj(init='epsg:27700') #british national grid
sql = "SELECT * FROM route WHERE route.originsiteid != '';"
df = pd.read_sql_query(sql,con)
for i in range(0,df.shape[0]): 
    route = df.iloc[i]
    originSiteID = route['originsiteid']
    destSiteID = route['destsiteid']
    #get O and D blueooth sensor site GPS
    sql = "SELECT * FROM sensorsite WHERE sensorID='%s'"%originSiteID
    df_o = pd.read_sql_query(sql,con)
    sql = "SELECT * FROM sensorsite WHERE sensorID='%s'"%destSiteID
    df_d = pd.read_sql_query(sql,con)
    if df_o.shape[0]==0:
        print "no data for "+originSiteID
        continue
    if df_d.shape[0]==0:
        print "no data for "+destSiteID
        continue       #if missing site info, skip this route
    o_easting = df_o['easting'][0]
    o_northing = df_o['northing'][0]
    d_easting = df_d['easting'][0]
    d_northing = df_d['northing'][0]
    #find nearest O and D road links to sensor locations 
    (o_lon,o_lat) = pyproj.transform(bng, wgs84, o_easting, o_northing) #project uses nonISO lonlat convention 
    (d_lon,d_lat) = pyproj.transform(bng, wgs84, d_easting, d_northing) #project uses nonISO lonlat convention 
    sql = "SELECT gid,name,ST_Distance(ways.the_geom,ST_SetSRID(ST_MakePoint(%f, %f),4326)) FROM ways ORDER BY 3 ASC LIMIT 1;"%(o_lon,o_lat)    #4326=SRID code for WGS84
    print sql
    df_on = pd.read_sql_query(sql,con)
    o_link_gid = df_on['gid'][0]
    sql = "SELECT gid,name,ST_Distance(ways.the_geom,ST_SetSRID(ST_MakePoint(%f, %f),4326)) FROM ways ORDER BY 3 ASC LIMIT 1;"%(d_lon,d_lat) 
    df_dn = pd.read_sql_query(sql,con)  #dest nearest link
    d_link_gid = df_dn['gid'][0]
    #print("routeID:%s, originSiteID:%s , destSiteID:%s , o_link_gid:%s, d_link_gid:%s  "%(route['routeid'],originSiteID,destSiteID, o_link_gid, d_link_gid))
    #find fastest route between them
    sql = "SELECT * FROM pgr_dijkstra('SELECT gid AS id, source, target, length AS cost FROM ways', %d,%d, directed := false), ways  WHERE ways.gid=pgr_dijkstra.edge;"%(o_link_gid, d_link_gid)
    print sql
    df_route = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='the_geom')
    #add a row of counts for each link in the route (at which time? TODO currently using origin time- interpolate?)
    for i in range(0,df_route.shape[0]):
        rc = df_route.iloc[i]                   #route component
        sql = "INSERT INTO routelink (routeid, link_gid) VALUES ('%s', %s);"%(route['routeid'], rc['gid'])
        print sql
        cur.execute(sql)
        con.commit()